In [71]:
from openai import OpenAI
from secret import OPENAI_API_KEY
import base64

In [72]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [73]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [74]:
def chat(base64_image):
    try:
        # Messages pour le contexte
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Décris moi cette image avec 5 mots-clés. Le format est le suivant : [mot-clé1, mot-clé2, mot-clé3, mot-clé4, mot-clé5]",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ]
            }
        ]

        chat_completion = client.chat.completions.create(
            model="gpt-4o-mini", messages=messages
        )

        return chat_completion.choices[0].message.content.strip()

    except Exception as e:
        print(f"Erreur OpenAI : {e}")

In [75]:
image_path = "test_data/20240826_184159.jpg"
base64_image  = encode_image(image_path)

In [76]:
result = chat(base64_image)

In [77]:
print(result)

[sentier, forêt, nature, feuilles, verdure]
